# Download Data for baseline

In [13]:
import wget
url = r"https://tickettagger.blob.core.windows.net/datasets/dataset-labels-top3-30k-real.txt"
output = r"../dataset/dataset-labels-top3-30k-real.txt"
wget.download(url,output)

'../dataset/dataset-labels-top3-30k-real (1).txt'

## look at pivot table of data from other repos

In [24]:
import pandas as pd

df = pd.read_csv("..\dataset\multiple_projects_benchmark_data.csv")
df.head()

,f0_,repo_name
0,__label__bug Single most recently added Custom...,https://api.github.com/repos/qgis/QGIS
1,__label__bug layer style effects not compatibl...,https://api.github.com/repos/qgis/QGIS
2,__label__bug QGIS 3.10.0 fails to launch on Wi...,https://api.github.com/repos/qgis/QGIS
3,__label__bug TIN interpolation in Model Builde...,https://api.github.com/repos/qgis/QGIS
4,__label__bug Map Tip Position / Follow the Hyp...,https://api.github.com/repos/qgis/QGIS


In [25]:
df = df.dropna()

In [26]:
df['label'] = df['f0_'].str.split(" ").map(lambda x: x[0])
df.head()

,f0_,repo_name,label
0,__label__bug Single most recently added Custom...,https://api.github.com/repos/qgis/QGIS,__label__bug
1,__label__bug layer style effects not compatibl...,https://api.github.com/repos/qgis/QGIS,__label__bug
2,__label__bug QGIS 3.10.0 fails to launch on Wi...,https://api.github.com/repos/qgis/QGIS,__label__bug
3,__label__bug TIN interpolation in Model Builde...,https://api.github.com/repos/qgis/QGIS,__label__bug
4,__label__bug Map Tip Position / Follow the Hyp...,https://api.github.com/repos/qgis/QGIS,__label__bug


In [29]:
df.groupby(['repo_name','label']).agg({'f0_':'count'})

f0_
repo_name                                          label                      
https://api.github.com/repos/Kaiserreich/Kaiser... __label__bug           7476
https://api.github.com/repos/Microsoft/vscode      __label__bug           8005
                                                   __label__question       170
https://api.github.com/repos/Regalis11/Barotrauma  __label__bug           3774
                                                   __label__question         7
https://api.github.com/repos/dbeaver/dbeaver       __label__bug           4246
                                                   __label__enhancement    467
                                                   __label__question      1395
https://api.github.com/repos/godotengine/godot     __label__bug          11591
                                                   __label__enhancement   1945
https://api.github.com/repos/hashicorp/terraform   __label__bug           5008
                                                   __label__enhancement   1531
                                                   __label__question       277
https://api.github.com/repos/libsdl-org/SDL        __label__bug           3183
                                                   __label__enhancement    196
https://api.github.com/repos/microsoft/vscode      __label__bug           8291
https://api.github.com/repos/qgis/QGIS             __label__bug           4612
https://api.github.com/repos/rstudio/rstudio       __label__bug           1997
                                                   __label__enhancement    757
https://api.github.com/repos/symfony/symfony       __label__bug           4045
                                                   __label__enhancement    120
https://api.github.com/repos/tgstation/tgstation   __label__bug           5575